<a href="https://colab.research.google.com/github/johnwiese/trading-crypto/blob/master/Crypto_Data_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Tools for Bitcoin data and simple fractal moving averages for entries and exits

Convert to tsv use  DataFrame.to_tsv() in pandas

In [58]:
import numpy as np       # Data formating
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import os.path
from os import path
import datetime
from datetime import date
import itertools
from pandas.io.json import json_normalize
!pip install coinbasepro



# Tools for getting data for Bitcoin


1.   Load historic data files
2.   Get live hourly data from Coinbase Pro API
3.   Convert to JSON https://datatofish.com/export-pandas-dataframe-json/



In [4]:
from google.colab import drive
drive.mount('/content/drive')

!pwd
!ls -al

%cd "./drive/My Drive/ML/assets/"
!ls -al
   



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ML/assets
total 8348
-rw------- 1 root root     151 Oct 10 00:25  Bitfinex_BTCUSD_d.csv.gsheet
-rw------- 1 root root 1424089 Oct 10 01:55 'Coinbase_BTCUSD_1h (1).csv'
-rw------- 1 root root 1424089 Oct 10 01:56 'Coinbase_BTCUSD_1h (2).csv'
-rw------- 1 root root 1424089 Oct 10 01:59 'Coinbase_BTCUSD_1h (3).csv'
-rw------- 1 root root 1424089 Oct 10 02:02 'Coinbase_BTCUSD_1h (4).csv'
-rw------- 1 root root 1424089 Oct 10 02:03 'Coinbase_BTCUSD_1h (5).csv'
-rw------- 1 root root 1424089 Oct 10 01:53  Coinbase_BTCUSD_1h.csv
-rw------- 1 root root     151 Oct 10 00:25  Coinbase_BTCUSD_1h.csv.gsheet
-rw------- 1 root root     151 Oct 10 17:32  Coinbase_BTCUSD_d.csv.gsheet
[Errno 2] No such file or directory: './drive/My Drive/ML/assets/'
/content/drive/My Drive/ML/assets
total 8348
-rw------- 1 root root     151 Oct 10 00:25  Bitfinex_BTCU

In [0]:
!pip install --upgrade --quiet gspread

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

**Get historic data from two sources**
  bitfinex 
    Has only day data but a lot starting back in 2015-02-09
  coinbase historic
    Has hourly data
Get real live hourly data from Coinbase using the Coinbase Pro API
  Read coinbase from the last point of last of the historic data

**Plan for charts **
  Load the bitinex daily data
  From the first hourly historic data from coinbase merge both historic data from bitfinx and coinbase historic data
  Note: Convert Coinbase to day data bars. One bar starts the 24 day bare from GTC midnight to the next GTC zero hour

Once the historic data is merged into one for display then start getting new hourly data from Coinbase every hour and then convert to daily bars


In [0]:
# Get historic data from bitfinex and coinbase
# bitfinex has a lot of day data

bitfinex_btcusd_file = 'Bitfinex_BTCUSD_d.csv'
coinbase_btcusd_file = 'Coinbase_BTCUSD_d.cvs'  # 'Coinbase_BTCUSD_1h.csv'
base_path = '/content/drive/My Drive/ML/assets'

bitfinex_btcusd_path = os.path.join(base_path,bitfinex_btcusd_file)

In [0]:
# get dataframes
# get bitfinex
worksheet_bitfinex = gc.open(bitfinex_btcusd_file).sheet1
# get_all_values gives a list of rows.
rows_bitfinex = worksheet_bitfinex.get_all_values()
df_bitfinex = pd.DataFrame.from_records(rows_bitfinex)




In [9]:
!pwd
!ls -al
print(os.path.exists("Coinbase_BTCUSD_d.csv.gsheet"))
# get coinbase
worksheet_coinbase = gc.open("Coinbase_BTCUSD_d.csv").sheet1
# get_all_values gives a list of rows.
rows_coinbase = worksheet_coinbase.get_all_values()
df_coinbase = pd.DataFrame.from_records(rows_coinbase)

/content/drive/My Drive/ML/assets
total 8348
-rw------- 1 root root     151 Oct 10 00:25  Bitfinex_BTCUSD_d.csv.gsheet
-rw------- 1 root root 1424089 Oct 10 01:55 'Coinbase_BTCUSD_1h (1).csv'
-rw------- 1 root root 1424089 Oct 10 01:56 'Coinbase_BTCUSD_1h (2).csv'
-rw------- 1 root root 1424089 Oct 10 01:59 'Coinbase_BTCUSD_1h (3).csv'
-rw------- 1 root root 1424089 Oct 10 02:02 'Coinbase_BTCUSD_1h (4).csv'
-rw------- 1 root root 1424089 Oct 10 02:03 'Coinbase_BTCUSD_1h (5).csv'
-rw------- 1 root root 1424089 Oct 10 01:53  Coinbase_BTCUSD_1h.csv
-rw------- 1 root root     151 Oct 10 00:25  Coinbase_BTCUSD_1h.csv.gsheet
-rw------- 1 root root     151 Oct 10 17:32  Coinbase_BTCUSD_d.csv.gsheet
True


In [10]:
print(df_bitfinex)

               0       1         2  ...         5           6            7
0           Date  Symbol      Open  ...     Close  Volume BTC   Volume USD
1     2019-10-08  BTCUSD      8227  ...   8205.14     1584.55  13097499.82
2     2019-10-07  BTCUSD    7882.9  ...      8227     8149.21  66182672.83
3     2019-10-06  BTCUSD    8162.5  ...    7882.9     5857.54  46641948.11
4     2019-10-05  BTCUSD      8170  ...    8162.5     3327.26  27021471.49
5     2019-10-04  BTCUSD      8258  ...      8170     3051.67   24946287.9
6     2019-10-03  BTCUSD   8414.88  ...      8258     3716.19  30623737.96
7     2019-10-02  BTCUSD    8334.9  ...   8414.88     3448.52  28570950.97
8     2019-10-01  BTCUSD   8331.14  ...    8334.9      5835.9  49049132.41
9     2019-09-30  BTCUSD      8090  ...   8331.14    10919.75  87839295.32
10    2019-09-29  BTCUSD    8238.2  ...      8090     6143.56  49454030.63
11    2019-09-28  BTCUSD    8213.4  ...    8238.2     5447.13  44602497.95
12    2019-09-27  BTCUSD 

In [11]:
print(df_coinbase)

               0       1         2  ...         5           6            7
0           Date  Symbol      Open  ...     Close  Volume BTC   Volume USD
1     2019-10-08  BTCUSD      8209  ...   8216.39        2329  19198358.02
2     2019-10-07  BTCUSD   7859.79  ...      8209     9969.38  80691163.82
3     2019-10-06  BTCUSD   8147.63  ...   7859.79     7770.16  61753052.16
4     2019-10-05  BTCUSD   8156.67  ...   8147.63     3809.86  30873507.65
5     2019-10-04  BTCUSD   8240.41  ...   8156.67     7620.06   62078561.3
6     2019-10-03  BTCUSD   8381.72  ...   8240.41     7550.73  62037926.73
7     2019-10-02  BTCUSD   8321.52  ...   8381.72     7006.37  57883644.17
8     2019-10-01  BTCUSD   8304.96  ...   8321.52     12133.1  101594010.6
9     2019-09-30  BTCUSD    8052.4  ...   8304.96    14235.43  114990553.1
10    2019-09-29  BTCUSD   8217.47  ...    8052.4     6911.51  55634607.52
11    2019-09-28  BTCUSD   8193.99  ...   8217.47     8846.18  72162358.51
12    2019-09-27  BTCUSD 

In [0]:
# Reverse the rows in the dataframe from oldest values to newest valude
df_coinbase_reverse = df_coinbase

df_coinbase_reverse = df_coinbase_reverse.iloc[::-1]

df_bitfinex_reverse = df_bitfinex.iloc[::-1]

In [13]:
# print reversed coinbase
print(df_coinbase_reverse)

               0       1         2  ...         5           6            7
1761  2014-12-01  BTCUSD       300  ...       370     0.05656        19.53
1760  2014-12-02  BTCUSD       370  ...       378       15.01      5675.07
1759  2014-12-03  BTCUSD       378  ...       378      0.5466       206.52
1758  2014-12-04  BTCUSD       378  ...     377.1        0.01         3.77
1757  2014-12-05  BTCUSD     377.1  ...     377.1           0            0
1756  2014-12-06  BTCUSD     377.1  ...       378       0.015         5.67
1755  2014-12-07  BTCUSD       378  ...       378           0            0
1754  2014-12-08  BTCUSD       378  ...       375       0.235        88.13
1753  2014-12-09  BTCUSD       375  ...       375           0            0
1752  2014-12-10  BTCUSD       375  ...     360.5        0.02         7.59
1751  2014-12-11  BTCUSD     360.5  ...     360.5           0            0
1750  2014-12-12  BTCUSD     360.5  ...       350        3.15      1115.49
1749  2014-12-13  BTCUSD 

In [14]:
# print reversed bitfinex
print(df_bitfinex_reverse)

               0       1         2  ...         5           6            7
1691  2015-02-09  BTCUSD    224.22  ...    220.61    29625.03   6493501.42
1690  2015-02-10  BTCUSD    220.61  ...    220.96    29268.95   6402350.57
1689  2015-02-11  BTCUSD    220.96  ...    219.19    19289.28   4268986.79
1688  2015-02-12  BTCUSD    219.19  ...     222.6    11190.65   2468883.37
1687  2015-02-13  BTCUSD    222.39  ...       236    29079.11   6806972.89
1686  2015-02-14  BTCUSD    236.13  ...    258.12    42984.94  10739014.48
1685  2015-02-15  BTCUSD       257  ...    234.14    71361.28   17384420.7
1684  2015-02-16  BTCUSD    234.14  ...    236.78    45750.85  10798751.12
1683  2015-02-17  BTCUSD    236.78  ...    242.87    42669.08  10228035.45
1682  2015-02-18  BTCUSD    242.87  ...     234.8    38588.38   9164141.83
1681  2015-02-19  BTCUSD     234.8  ...     241.9    22792.55   5446200.04
1680  2015-02-20  BTCUSD     241.9  ...    245.53    33337.97   8136342.67
1679  2015-02-21  BTCUSD 

In [0]:
# Convert to tsv
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html


In [0]:
import coinbasepro as cbp

In [16]:
client = cbp.PublicClient()
client.get_product_ticker('BTC-USD')


{'ask': Decimal('8311.99'),
 'bid': Decimal('8311.03'),
 'price': Decimal('8311.03'),
 'size': Decimal('0.02272793'),
 'time': datetime.datetime(2019, 10, 11, 23, 32, 0, 800000),
 'trade_id': 75843854,
 'volume': Decimal('12139.18013373')}

The granularity field must be one of the following values: {60, 300, 900, 3600, 21600, 86400}. Otherwise, your request will be rejected. These values correspond to timeslices representing one minute, five minutes, fifteen minutes, one hour, six hours, and one day, respectively.

In [41]:
next_coinbase_date_after_historic = '2019-10-09T00:00:00'
d_iso = datetime.datetime(year = 2019, month = 10, day = 9).isoformat()

print('ISO date now: ' + d_iso)

ISO date now: 2019-10-09T00:00:00


In [59]:


client.get_product_trades('BTC-USD')
client.get_product_24hr_stats('BTC-USD')
#client.get_product_historic_rates('BTC-USD',start='2019-07-01T00:00:00',granularity=3600) # hourly bars

historic_btcusd_json = client.get_product_historic_rates('BTC-USD',start=d_iso,granularity=86400) # daily bars


df_coinbase_json = json_normalize(historic_btcusd_json)
df_coinbase_csv = df_coinbase_json.to_csv(sep='\t', encoding='utf-8')


print(df_coinbase_csv)



	close	high	low	open	time	volume
0	8287.58	8313.92	8261.46	8263.02	2019-10-12	232.67290412
1	8267.33	8826	8226	8587.49	2019-10-11	12446.84471206
2	8587.5	8660	8452.5	8590.01	2019-10-10	7946.64222782
3	8590	8712.45	8121	8180	2019-10-09	14459.7776891
4	8180	8342.97	8107.26	8208.99	2019-10-08	5857.54021433
5	8209	8310.57	7762.35	7859.79	2019-10-07	10012.00725014
6	7859.79	8176.18	7780.85	8147.63	2019-10-06	8697.93975986
7	8147.63	8200	8020.2	8156.68	2019-10-05	3826.82870697
8	8156.67	8241.35	8002.21	8241.27	2019-10-04	7715.28010237
9	8240.41	8419.95	8059.22	8381.72	2019-10-03	7778.11842343
10	8381.72	8391	8168.85	8321.87	2019-10-02	7040.87668343
11	8321.52	8535	8200	8304.95	2019-10-01	12266.35796784
12	8304.96	8393.23	7701	8054.09	2019-09-30	15142.61949966
13	8052.4	8246.87	7902.96	8217.47	2019-09-29	7142.36916333
14	8217.47	8343	8015.25	8193.99	2019-09-28	9039.55200543
15	8193.99	8290	7861.02	8060	2019-09-27	15914.66412076
16	8060	8464.08	7712.45	8438.35	2019-09-26	27664.78945578
17	8438